## AIM: Clean the data mined from MLB statcast and combined the separate .csv files into one file that will be used for modeling.

***

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

For the purpose of time, I will be walking through the inital data cleaning with only this one .csv file from the 2016 season. 

Since the .csv files contain close to 1 million or greater rows, they take a long time to read in. In addition, the process was identical for ALL raw pitch data .csv files and so running through multiple would be redundant for the purpose of illustrating this process.

Eventually, all .csvs were combined and more data processing was done on the combined file.

In [22]:
# Read in the .csv file
df = pd.read_csv('pitches16.csv', index_col = 0)

In [23]:
# Get a peek
df.head()

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,0,CU,2016-11-02,76.6,2.7233,6.3880,Mike Montgomery,492841.0,543557.0,field_out,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard
1,1,CU,2016-11-02,74.3,2.8223,6.2811,Mike Montgomery,492841.0,543557.0,NaN,...,7.0,8.0,7.0,8.0,8.0,7.0,7.0,8.0,Standard,Standard
2,2,FF,2016-11-02,94.9,-1.4961,6.0764,Carl Edwards Jr.,434658.0,605218.0,single,...,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard
3,3,FF,2016-11-02,95.6,-1.4613,6.0870,Carl Edwards Jr.,434658.0,605218.0,NaN,...,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard
4,4,FF,2016-11-02,94.8,-1.3957,5.9541,Carl Edwards Jr.,446386.0,605218.0,walk,...,6.0,8.0,6.0,8.0,8.0,6.0,6.0,8.0,Standard,Standard


In [24]:
# Don't know why the index col won't get read in as index
df.drop(['index'], axis = 1, inplace = True)

In [25]:
# See the shape
df.shape

(726023, 89)

In [26]:
# Look at the columns, start to see which ones can be dropped immediately
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [27]:
# Check nulls
df.isna().sum()

pitch_type                 919
game_date                    0
release_speed              102
release_pos_x             8179
release_pos_z             8179
                         ...  
post_home_score              0
post_bat_score               0
post_fld_score               0
if_fielding_alignment    13238
of_fielding_alignment    13238
Length: 89, dtype: int64

There are a lot of NaN values in the data. Not all NaN values are equal. Some represent missing data while some likely represent 0.

In addition, some can be dropped immediately as there is no use of the column or row. I will begin by dropping rows containing NaNs in critical columns as they can't be used for downstream purposes.

In [28]:
# Drop rows with no pitch type data so drop, usually also no speed data or position data
df.dropna(subset=['pitch_type'], inplace = True)

# No position data for pitches missing speed either
df.dropna(subset=['release_speed'], inplace = True)

# Field shifts not important for scope of my project
df.drop(['if_fielding_alignment', 'of_fielding_alignment'], axis = 1, inplace = True)

# No release position means no zone info, therefore can't be used to predict pitch location
df.dropna(subset=['release_pos_x'], inplace = True)

In [29]:
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

A lot of these columns have a very unclear and "baseball statsy" title such as `woba_value` and `vx0`.

To get a definition of these columns, I referred to: https://baseballsavant.mlb.com/csv-docs 

As such, I dropped many columns after referring to this documentation and based on the column description, I decided they were not relevant to my project.

In [30]:
# Start dropping unneeded columns
df.drop(['spin_dir', 'release_pos_x', 'release_pos_z', 'spin_rate_deprecated', 'break_angle_deprecated', 'home_team', 
         'away_team', 'hit_location', 'game_year', 'pfx_x', 'pfx_z', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 
         'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'hit_distance_sc', 
         'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk'], 
        axis =1 , inplace = True)

# Start iloc dropping bulk clustered unneeded columns at the end (post pitch-scores)
df = df.iloc[:,:-4]

# Keep dropping unneeded columns
df.drop(['pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9','estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
       'woba_value', 'woba_denom', 'babip_value', 'iso_value',
       'launch_speed_angle', 'home_score', 'away_score'], 
        axis =1 , inplace = True)

In [31]:
# Check current columns
df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'player_name', 'batter',
       'pitcher', 'events', 'description', 'break_length_deprecated', 'zone',
       'des', 'game_type', 'stand', 'p_throws', 'type', 'bb_type', 'balls',
       'strikes', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'sz_top', 'sz_bot', 'release_pos_y',
       'at_bat_number', 'pitch_number', 'pitch_name', 'bat_score',
       'fld_score'],
      dtype='object')

In [32]:
# Check current NaN situations
df.isna().sum()

pitch_type                      0
game_date                       0
release_speed                   0
player_name                     0
batter                          0
pitcher                         0
events                     532006
description                     0
break_length_deprecated    717030
zone                            0
des                        532007
game_type                       0
stand                           0
p_throws                        0
type                            0
bb_type                    588095
balls                           0
strikes                         0
plate_x                         0
plate_z                         0
on_3b                      647707
on_2b                      583538
on_1b                      500962
outs_when_up                    0
inning                          0
sz_top                          0
sz_bot                          0
release_pos_y                   0
at_bat_number                   0
pitch_number  

There is a LOT of NaNs for the columns encoding information about whether somebody is on 1st/2nd/3rd base. These are likely just representing a value of 0 (ie. nobody on base) given the large number and the distribution of values (greatest NaNs to least is 3rd, 2nd and 1st, representing the difficulties in reaching those respective bases).

In [33]:
# Check exactly what the on_3b base is holding in terms of values
# The other columns will likely be identical
df['on_3b'].value_counts()

592178.0    419
543401.0    410
518626.0    406
514888.0    400
542340.0    383
           ... 
547004.0      1
598284.0      1
602922.0      1
518560.0      1
488912.0      1
Name: on_3b, Length: 703, dtype: int64

From the value counts, it looks like the `on_3b` column is encoding the player ID that is currently on 3rd base. The other base runner columns are sure to be the same. Given this information, it is very likely that the NaNs in this column correspond to nobody on base and therefore can be filled in with 0.

In [34]:
# Fill NaNs in base columns with 0 to indicate nobody is on the base
df.fillna({'on_3b':0, 'on_2b': 0, 'on_1b':0}, inplace = True)

# Batted ball in play not relevant to scope of capstone
df.drop(['bb_type'], axis = 1, inplace = True)

# Description is too specific for purpose of capstone
df.drop(['des'], axis = 1, inplace = True)

# Do not need the degree of break of the pitch
df.drop(['break_length_deprecated'], axis = 1, inplace = True)

In [36]:
# Description and events sound like they mean similar things
# Do a value_counts to see what values they hold
print(df['events'].value_counts())
print('')
print(df['description'].value_counts())

field_out                       74450
strikeout                       38903
single                          27532
walk                            14104
double                           8242
home_run                         5622
force_out                        3868
grounded_into_double_play        3720
hit_by_pitch                     1651
field_error                      1581
sac_fly                          1184
sac_bunt                         1013
triple                            866
intent_walk                       849
double_play                       442
fielders_choice_out               287
caught_stealing_2b                228
strikeout_double_play             141
fielders_choice                    96
pickoff_caught_stealing_2b         48
other_out                          41
catcher_interf                     40
pickoff_1b                         33
sac_fly_double_play                24
caught_stealing_3b                 18
pickoff_2b                         13
run         

In [37]:
# Description is good enough for my purposes
# Events is too detailed
df.drop(['events'], axis = 1, inplace = True)

In [39]:
# Check the progress, only 264 left in the pitch names column
df.isna().sum()

pitch_type         0
game_date          0
release_speed      0
player_name        0
batter             0
pitcher            0
description        0
zone               0
game_type          0
stand              0
p_throws           0
type               0
balls              0
strikes            0
plate_x            0
plate_z            0
on_3b              0
on_2b              0
on_1b              0
outs_when_up       0
inning             0
sz_top             0
sz_bot             0
release_pos_y      0
at_bat_number      0
pitch_number       0
pitch_name       264
bat_score          0
fld_score          0
dtype: int64

In [42]:
# SPECIFIC for the 2016 data
# Take a look at the remaining NaNs
# All from the same game it seems (weird), just drop
display(df[df['pitch_name'].isna()])
print('')
print(df[df['pitch_name'].isna()]['game_date'].value_counts())

,pitch_type,game_date,release_speed,player_name,batter,pitcher,description,zone,game_type,stand,...,outs_when_up,inning,sz_top,sz_bot,release_pos_y,at_bat_number,pitch_number,pitch_name,bat_score,fld_score
661528,160421_210811,2016-04-21,96.2,Danny Duffy,592206.0,518633.0,hit_into_play,12.0,R,R,...,2.0,9.0,3.78,1.78,54.1279,69.0,4.0,NaN,0.0,4.0
661529,160421_210739,2016-04-21,96.5,Danny Duffy,592206.0,518633.0,foul,14.0,R,R,...,2.0,9.0,3.78,1.78,54.0158,69.0,3.0,NaN,0.0,4.0
661530,160421_210715,2016-04-21,95.6,Danny Duffy,592206.0,518633.0,called_strike,4.0,R,R,...,2.0,9.0,3.65,1.83,53.8008,69.0,2.0,NaN,0.0,4.0
661531,160421_210655,2016-04-21,96.1,Danny Duffy,592206.0,518633.0,ball,13.0,R,R,...,2.0,9.0,3.72,1.90,53.7659,69.0,1.0,NaN,0.0,4.0
661532,160421_210609,2016-04-21,95.2,Danny Duffy,502110.0,518633.0,hit_into_play,6.0,R,R,...,1.0,9.0,3.78,1.69,53.9452,68.0,1.0,NaN,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
661787,160421_181740,2016-04-21,93.7,Edinson Volquez,408234.0,450172.0,called_strike,9.0,R,R,...,2.0,1.0,3.65,1.85,54.2603,3.0,2.0,NaN,0.0,0.0
661788,160421_181723,2016-04-21,94.8,Edinson Volquez,408234.0,450172.0,ball,7.0,R,R,...,2.0,1.0,3.65,1.81,54.3506,3.0,1.0,NaN,0.0,0.0
661789,160421_181641,2016-04-21,93.3,Edinson Volquez,457708.0,450172.0,hit_into_play,8.0,R,R,...,1.0,1.0,3.59,1.61,54.3718,2.0,1.0,NaN,0.0,0.0
661790,160421_181555,2016-04-21,92.8,Edinson Volquez,435079.0,450172.0,hit_into_play,1.0,R,R,...,0.0,1.0,3.41,1.53,54.4711,1.0,2.0,NaN,0.0,0.0



2016-04-21    264
Name: game_date, dtype: int64


In [43]:
# Drop
df.dropna(subset=['pitch_name'], inplace = True)

In [44]:
# All NaN's filled in
df.isna().sum()

pitch_type       0
game_date        0
release_speed    0
player_name      0
batter           0
pitcher          0
description      0
zone             0
game_type        0
stand            0
p_throws         0
type             0
balls            0
strikes          0
plate_x          0
plate_z          0
on_3b            0
on_2b            0
on_1b            0
outs_when_up     0
inning           0
sz_top           0
sz_bot           0
release_pos_y    0
at_bat_number    0
pitch_number     0
pitch_name       0
bat_score        0
fld_score        0
dtype: int64

In [45]:
# Since I dropped rows, it's best to reset the index
df.reset_index(drop = True, inplace = True)

# Turn into binary instead of having player IDs
df['on_3b'] = np.where(df['on_3b'] == 0, 0, 1)
df['on_2b'] = np.where(df['on_2b'] == 0, 0, 1)
df['on_1b'] = np.where(df['on_1b'] == 0, 0, 1)

# Binarize regular season/playoff games by creating effectively, dummy columns
df['reg_season'] = np.where(df['game_type'] == 'R', 1, 0)
df['post_season'] = np.where(np.isin(df['game_type'], ['F','D','L','W']), 1, 0)

# Drop the original game types columns now
df.drop(['game_type'], axis = 1, inplace = True)

# Binarize pitcher handedness and batter handedness
df['p_left'] = np.where(df['p_throws']=='L', 1, 0)
df['p_right'] = np.where(df['p_throws']=='R', 1, 0)

df['bat_left'] = np.where(df['stand']=='L', 1, 0)
df['bat_right'] = np.where(df['stand']=='R', 1, 0)

# Make sure everything adds up and nothing went wrong, columns should be inverse of their respective partner
print(df['p_right'].value_counts())
print(df['p_left'].value_counts())
print(df['bat_left'].value_counts())
print(df['bat_right'].value_counts())

1    529466
0    187300
Name: p_right, dtype: int64
0    529466
1    187300
Name: p_left, dtype: int64
0    417619
1    299147
Name: bat_left, dtype: int64
1    417619
0    299147
Name: bat_right, dtype: int64


In [46]:
# Drop the original columns now
df.drop(['p_throws'], axis = 1, inplace = True)

df.drop(['stand'], axis = 1, inplace = True)

df.drop(['release_pos_y'], axis = 1, inplace = True)

In [ ]:
# I decided not to drop the at_bat_number just yet because I need it to sort
# df.drop(['at_bat_number'], axis = 1, inplace = True)

In [47]:
# Sort by date and within date, sort by player name and then sort by chronological order of in game at bats and pitch number
df.sort_values(by = ['game_date', 'player_name', 'at_bat_number', 'pitch_number'], inplace = True, ignore_index = True)

In [48]:
# Check if the balls and strikes order makes sense (ie. should be increasing by 1 for the same at bat and reset for next at bat)
df.head(25)[['balls', 'strikes', 'at_bat_number']]

,balls,strikes,at_bat_number
0,0.0,0.0,4.0
1,0.0,1.0,4.0
2,0.0,0.0,5.0
3,0.0,1.0,5.0
4,1.0,1.0,5.0
5,1.0,2.0,5.0
6,0.0,0.0,6.0
7,1.0,0.0,6.0
8,1.0,1.0,6.0
9,2.0,1.0,6.0


In [ ]:
# Save to a new .csv file to preserve original in case any edits/new things need to be done on it
# df.to_csv('pitches_clean16.csv')

***

## Workflow for the pitch data from the 2013-2015 seasons

In [49]:
# df = pd.read_csv('pitches13-15.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (89,90) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [50]:
# Don't know why the index col won't get read in as index
# df.drop(['index'], axis = 1, inplace = True)

In [51]:
# df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,FF,2015-11-01,96.1,-2.1687,6.1653,Wade Davis,527038.0,451584.0,strikeout,called_strike,...,2.0,7.0,2.0,7.0,7.0,2.0,2.0,7.0,Standard,Strategic
1,FC,2015-11-01,93.1,-1.8205,6.2651,Wade Davis,527038.0,451584.0,NaN,foul,...,2.0,7.0,2.0,7.0,7.0,2.0,2.0,7.0,Standard,Strategic
2,FF,2015-11-01,97.0,-1.8044,6.2500,Wade Davis,527038.0,451584.0,NaN,foul,...,2.0,7.0,2.0,7.0,7.0,2.0,2.0,7.0,Standard,Strategic
3,FC,2015-11-01,93.6,-1.7449,6.1657,Wade Davis,527038.0,451584.0,NaN,ball,...,2.0,7.0,2.0,7.0,7.0,2.0,2.0,7.0,Strategic,Strategic
4,FF,2015-11-01,97.1,-1.8227,6.2320,Wade Davis,527038.0,451584.0,NaN,foul,...,2.0,7.0,2.0,7.0,7.0,2.0,2.0,7.0,Strategic,Standard


In [52]:
# df['game_date'].sort_values()

2147846    2013-03-31
2147674    2013-03-31
2147673    2013-03-31
2147672    2013-03-31
2147671    2013-03-31
              ...    
240        2015-11-01
241        2015-11-01
242        2015-11-01
222        2015-11-01
0          2015-11-01
Name: game_date, Length: 2147847, dtype: object

In [53]:
# df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [54]:
# df.isna().sum()

pitch_type                  3424
game_date                      0
release_speed               2761
release_pos_x            1445621
release_pos_z            1445621
                          ...   
post_home_score                0
post_bat_score                 0
post_fld_score                 0
if_fielding_alignment    1462555
of_fielding_alignment    1462555
Length: 89, dtype: int64

In [55]:
# # No pitch type data so drop, usually also no speed data or position data
# df.dropna(subset=['pitch_type'], inplace = True)

# # No position data for pitches missing speed either
# df.dropna(subset=['release_speed'], inplace = True)

# # Field shifts not important for scope of my project
# df.drop(['if_fielding_alignment', 'of_fielding_alignment'], axis = 1, inplace = True)

# # No release position means no zone info, therefore can't be used to predict pitch location
# df.dropna(subset=['release_pos_x'], inplace = True)

In [56]:
# # Start dropping unneeded columns
# df.drop(['spin_dir', 'release_pos_x', 'release_pos_z', 'spin_rate_deprecated', 'break_angle_deprecated', 'home_team', 
#          'away_team', 'hit_location', 'game_year', 'pfx_x', 'pfx_z', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 
#          'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'hit_distance_sc', 
#          'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk'], 
#         axis =1 , inplace = True)

# # Start iloc dropping bulk clustered unneeded columns at the end
# df = df.iloc[:,:-4]

# # Keep dropping unneeded columns
# df.drop(['pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
#        'fielder_7', 'fielder_8', 'fielder_9','estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
#        'woba_value', 'woba_denom', 'babip_value', 'iso_value',
#        'launch_speed_angle', 'home_score', 'away_score'], 
#         axis =1 , inplace = True)

In [57]:
# df.fillna({'on_3b':0, 'on_2b': 0, 'on_1b':0}, inplace = True)

# # Batted ball in play not relevant to scope of capstone
# df.drop(['bb_type'], axis = 1, inplace = True)

# # Description is too specific for purpose of capstone
# df.drop(['des'], axis = 1, inplace = True)

# df.drop(['break_length_deprecated'], axis = 1, inplace = True)

# df['description'].value_counts()

# # Description is good enough for my purposes
# df.drop(['events'], axis = 1, inplace = True)

In [58]:
# # All NaN's filled in
# df.isna().sum()

pitch_type       0
game_date        0
release_speed    0
player_name      0
batter           0
pitcher          0
description      0
zone             0
game_type        0
stand            0
p_throws         0
type             0
balls            0
strikes          0
plate_x          0
plate_z          0
on_3b            0
on_2b            0
on_1b            0
outs_when_up     0
inning           0
sz_top           0
sz_bot           0
release_pos_y    0
at_bat_number    0
pitch_number     0
pitch_name       0
bat_score        0
fld_score        0
dtype: int64

In [ ]:
# df.reset_index(drop = True, inplace = True)

# # Turn into binary
# df['on_3b'] = np.where(df['on_3b'] == 0, 0, 1)
# df['on_2b'] = np.where(df['on_2b'] == 0, 0, 1)
# df['on_1b'] = np.where(df['on_1b'] == 0, 0, 1)

# # Binarize regular season/playoff games
# df['reg_season'] = np.where(df['game_type'] == 'R', 1, 0)

# df['post_season'] = np.where(np.isin(df['game_type'], ['F','D','L','W']), 1, 0)

# # Drop the original game types columns now
# df.drop(['game_type'], axis = 1, inplace = True)

# # Binarize pitcher handedness and batter handedness
# df['p_left'] = np.where(df['p_throws']=='L', 1, 0)
# df['p_right'] = np.where(df['p_throws']=='R', 1, 0)

# df['bat_left'] = np.where(df['stand']=='L', 1, 0)
# df['bat_right'] = np.where(df['stand']=='R', 1, 0)

# print(df['p_right'].value_counts())
# print(df['p_left'].value_counts())
# print(df['bat_left'].value_counts())
# print(df['bat_right'].value_counts())

In [ ]:
# # Drop the original column now
# df.drop(['p_throws'], axis = 1, inplace = True)

# df.drop(['stand'], axis = 1, inplace = True)

# df.drop(['release_pos_y'], axis = 1, inplace = True)

In [ ]:
# df.drop(['at_bat_number'], axis = 1, inplace = True)

In [ ]:
# Sort by date and within date, sort by player name and then sort by chronological order of in game at bats and pitch number
# df.sort_values(by = ['game_date', 'player_name', 'at_bat_number', 'pitch_number'], inplace = True, ignore_index = True)

In [ ]:
# df.head(25)[['balls', 'strikes', 'at_bat_number']]

In [ ]:
# df['game_date'].sort_values()

In [ ]:
# df.to_csv('pitches_clean15.csv')

***

## Same workflow for data from the 2017-2019 seasons

In [59]:
# Data is from the 2017-2019 seasons
# df = pd.read_csv('pitches.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [60]:
# Don't know why the index col won't get read in as index
# df.drop(['index'], axis = 1, inplace = True)

In [61]:
# df.head()

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,SL,2019-10-30,87.9,-2.6489,5.5022,Daniel Hudson,488726.0,543339.0,strikeout,swinging_strike,...,2.0,6.0,2.0,6.0,6.0,2.0,2.0,6.0,Infield shift,Standard
1,FF,2019-10-30,95.9,-2.7721,5.5161,Daniel Hudson,488726.0,543339.0,NaN,foul,...,2.0,6.0,2.0,6.0,6.0,2.0,2.0,6.0,Infield shift,Standard
2,FF,2019-10-30,96.5,-2.6782,5.4191,Daniel Hudson,488726.0,543339.0,NaN,ball,...,2.0,6.0,2.0,6.0,6.0,2.0,2.0,6.0,Infield shift,Standard
3,FF,2019-10-30,96.0,-2.6479,5.5516,Daniel Hudson,488726.0,543339.0,NaN,foul,...,2.0,6.0,2.0,6.0,6.0,2.0,2.0,6.0,Infield shift,Standard
4,SL,2019-10-30,86.7,-2.7268,5.5882,Daniel Hudson,488726.0,543339.0,NaN,ball,...,2.0,6.0,2.0,6.0,6.0,2.0,2.0,6.0,Infield shift,Standard


In [62]:
# df.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [63]:
# df.isna().sum()

pitch_type                8040
game_date                    0
release_speed             9890
release_pos_x            10142
release_pos_z            10142
                         ...  
post_home_score              0
post_bat_score               0
post_fld_score               0
if_fielding_alignment    10381
of_fielding_alignment    10381
Length: 89, dtype: int64

In [64]:
# # No pitch type data so drop, usually also no speed data or position data
# df.dropna(subset=['pitch_type'], inplace = True)

# # No position data for pitches missing speed either
# df.dropna(subset=['release_speed'], inplace = True)

# # Field shifts not important for scope of my project
# df.drop(['if_fielding_alignment', 'of_fielding_alignment'], axis = 1, inplace = True)

# # No release position means no zone info, therefore can't be used to predict pitch location
# df.dropna(subset=['release_pos_x'], inplace = True)

In [65]:
# # Start dropping unneeded columns
# df.drop(['spin_dir', 'release_pos_x', 'release_pos_z', 'spin_rate_deprecated', 'break_angle_deprecated', 'home_team', 
#          'away_team', 'hit_location', 'game_year', 'pfx_x', 'pfx_z', 'inning_topbot', 'hc_x', 'hc_y', 'tfs_deprecated', 
#          'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'hit_distance_sc', 
#          'launch_speed', 'launch_angle', 'effective_speed', 'release_spin_rate', 'release_extension', 'game_pk'], 
#         axis =1 , inplace = True)

# # Start iloc dropping bulk clustered unneeded columns at the end
# df = df.iloc[:,:-4]

# # Keep dropping unneeded columns
# df.drop(['pitcher.1', 'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
#        'fielder_7', 'fielder_8', 'fielder_9','estimated_ba_using_speedangle', 'estimated_woba_using_speedangle',
#        'woba_value', 'woba_denom', 'babip_value', 'iso_value',
#        'launch_speed_angle', 'home_score', 'away_score'], 
#         axis =1 , inplace = True)

In [66]:
# df.fillna({'on_3b':0, 'on_2b': 0, 'on_1b':0}, inplace = True)

# # Batted ball in play not relevant to scope of capstone
# df.drop(['bb_type'], axis = 1, inplace = True)

# # Description is too specific for purpose of capstone
# df.drop(['des'], axis = 1, inplace = True)

# df.drop(['break_length_deprecated'], axis = 1, inplace = True)

# df['description'].value_counts()

# # Description is good enough for my purposes
# df.drop(['events'], axis = 1, inplace = True)

In [67]:
# All NaN's filled in
# df.isna().sum()

pitch_type       0
game_date        0
release_speed    0
player_name      0
batter           0
pitcher          0
description      0
zone             0
game_type        0
stand            0
p_throws         0
type             0
balls            0
strikes          0
plate_x          0
plate_z          0
on_3b            0
on_2b            0
on_1b            0
outs_when_up     0
inning           0
sz_top           0
sz_bot           0
release_pos_y    0
at_bat_number    0
pitch_number     0
pitch_name       0
bat_score        0
fld_score        0
dtype: int64

In [ ]:
# df.reset_index(drop = True, inplace = True)

# # Turn into binary
# df['on_3b'] = np.where(df['on_3b'] == 0, 0, 1)
# df['on_2b'] = np.where(df['on_2b'] == 0, 0, 1)
# df['on_1b'] = np.where(df['on_1b'] == 0, 0, 1)

# # Binarize regular season/playoff games
# df['reg_season'] = np.where(df['game_type'] == 'R', 1, 0)

# df['post_season'] = np.where(np.isin(df['game_type'], ['F','D','L','W']), 1, 0)

# # Drop the original game types columns now
# df.drop(['game_type'], axis = 1, inplace = True)

# # Binarize pitcher handedness and batter handedness
# df['p_left'] = np.where(df['p_throws']=='L', 1, 0)
# df['p_right'] = np.where(df['p_throws']=='R', 1, 0)

# df['bat_left'] = np.where(df['stand']=='L', 1, 0)
# df['bat_right'] = np.where(df['stand']=='R', 1, 0)

# print(df['p_right'].value_counts())
# print(df['p_left'].value_counts())
# print(df['bat_left'].value_counts())
# print(df['bat_right'].value_counts())

In [ ]:
# # Drop the original column now
# df.drop(['p_throws'], axis = 1, inplace = True)

# df.drop(['stand'], axis = 1, inplace = True)

# df.drop(['release_pos_y'], axis = 1, inplace = True)

In [ ]:
# df.drop(['at_bat_number'], axis = 1, inplace = True)

In [ ]:
# Sort by date and within date, sort by player name and then sort by chronological order of in game at bats and pitch number
# df.sort_values(by = ['game_date', 'player_name', 'at_bat_number', 'pitch_number'], inplace = True, ignore_index = True)

In [ ]:
# df.head(25)[['balls', 'strikes', 'at_bat_number']]

In [ ]:
# df['game_date'].sort_values()

In [ ]:
# df.to_csv('pitches_clean17-19.csv')

***

## MERGING

Here I will now combine the 3 separate .csv files into one master file.

In [68]:
# Read each of them in as their own dataframe

pitch_1719 = pd.read_csv('pitches_clean17-19.csv', index_col=0)

pitch_1315 = pd.read_csv('pitches_clean13-15.csv', index_col=0)

pitch16 = pd.read_csv('pitches_clean16.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [69]:
# Concatenate them row wise
merged = pd.concat([pitch_1315, pitch16, pitch_1719], axis=0, ignore_index = True, sort = False)

In [70]:
# Check that the date range is what I want
merged['game_date']

0          2015-04-05
1          2015-04-05
2          2015-04-05
3          2015-04-05
4          2015-04-05
              ...    
3615432    2019-10-30
3615433    2019-10-30
3615434    2019-10-30
3615435    2019-10-30
3615436    2019-10-30
Name: game_date, Length: 3615437, dtype: object

In [71]:
# Order should be good to go as is given how they were concatenated, no need to sort values
# Recall that they were sorted prior to being saved as clean .csv files
merged.reset_index(drop = True, inplace = True)
merged

,pitch_type,game_date,release_speed,player_name,batter,pitcher,description,zone,type,balls,...,pitch_number,pitch_name,bat_score,fld_score,reg_season,post_season,p_left,p_right,bat_left,bat_right
0,SI,2015-04-05,90.1,Adam Wainwright,451594.0,425794.0,ball,13.0,B,0.0,...,1.0,Sinker,0.0,1.0,1,0,0,1,1,0
1,FC,2015-04-05,88.1,Adam Wainwright,451594.0,425794.0,hit_into_play_no_out,5.0,X,1.0,...,2.0,Cutter,0.0,1.0,1,0,0,1,1,0
2,FF,2015-04-05,92.2,Adam Wainwright,624585.0,425794.0,foul,5.0,S,0.0,...,1.0,4-Seam Fastball,0.0,1.0,1,0,0,1,0,1
3,CU,2015-04-05,76.6,Adam Wainwright,624585.0,425794.0,blocked_ball,14.0,B,0.0,...,2.0,Curveball,0.0,1.0,1,0,0,1,0,1
4,FC,2015-04-05,89.9,Adam Wainwright,624585.0,425794.0,hit_into_play,9.0,X,1.0,...,4.0,Cutter,0.0,1.0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615432,FF,2019-10-30,89.1,Zack Greinke,665742.0,425844.0,ball,11.0,B,0.0,...,1.0,4-Seam Fastball,1.0,2.0,0,1,0,1,1,0
3615433,CU,2019-10-30,71.1,Zack Greinke,665742.0,425844.0,swinging_strike,11.0,S,1.0,...,2.0,Curveball,1.0,2.0,0,1,0,1,1,0
3615434,CH,2019-10-30,88.3,Zack Greinke,665742.0,425844.0,ball,13.0,B,1.0,...,3.0,Changeup,1.0,2.0,0,1,0,1,1,0
3615435,CH,2019-10-30,88.0,Zack Greinke,665742.0,425844.0,ball,8.0,B,2.0,...,4.0,Changeup,1.0,2.0,0,1,0,1,1,0


In [72]:
# Again, double check that the order of the rows make sense in terms of baseball chronological order
# Each at bat number should start with 0 strikes and 0 balls
# The new row will have +1 in either strikes OR balls and should correlated with previous row description
# New at bat number will reset balls/strikes to 0
# Looks good for the most part, can't spot any striking errors
merged.head(25)[['balls', 'strikes', 'description', 'at_bat_number']]

,balls,strikes,description,at_bat_number
0,0.0,0.0,ball,6.0
1,1.0,0.0,hit_into_play_no_out,6.0
2,0.0,0.0,foul,7.0
3,0.0,1.0,blocked_ball,7.0
4,1.0,2.0,hit_into_play,7.0
5,0.0,0.0,blocked_ball,8.0
6,1.0,0.0,swinging_strike,8.0
7,1.0,1.0,called_strike,8.0
8,1.0,2.0,blocked_ball,8.0
9,2.0,2.0,called_strike,8.0


In [327]:
# Not sure if I should drop yet
# merged.drop(['at_bat_number'], axis = 1, inplace = True)

In [328]:
# Save to a final .csv file
# merged.to_csv('pitches_finalv2.csv')

***

## Data Processing / Feature Engineering
***

In [73]:
# Read in the combined dataframe (is pretty large so will take some time)
df = pd.read_csv('pitches_finalv2.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [74]:
# Start converting needless float columns to int to reduce file size
df['zone'] = df['zone'].astype('int')

df['balls'] = df['balls'].astype('int')
df['strikes'] = df['strikes'].astype('int')

df['bat_score'] = df['bat_score'].astype('int')
df['fld_score'] = df['fld_score'].astype('int')

df['pitch_number'] = df['pitch_number'].astype('int')
df['inning'] = df['inning'].astype('int')
df['outs_when_up'] = df['outs_when_up'].astype('int')


In [75]:
# Check everything is still alright
df.head()

,pitch_type,game_date,release_speed,player_name,batter,pitcher,description,zone,type,balls,...,pitch_number,pitch_name,bat_score,fld_score,reg_season,post_season,p_left,p_right,bat_left,bat_right
0,SI,2015-04-05,90.1,Adam Wainwright,451594.0,425794.0,ball,13,B,0,...,1,Sinker,0,1,1,0,0,1,1,0
1,FC,2015-04-05,88.1,Adam Wainwright,451594.0,425794.0,hit_into_play_no_out,5,X,1,...,2,Cutter,0,1,1,0,0,1,1,0
2,FF,2015-04-05,92.2,Adam Wainwright,624585.0,425794.0,foul,5,S,0,...,1,4-Seam Fastball,0,1,1,0,0,1,0,1
3,CU,2015-04-05,76.6,Adam Wainwright,624585.0,425794.0,blocked_ball,14,B,0,...,2,Curveball,0,1,1,0,0,1,0,1
4,FC,2015-04-05,89.9,Adam Wainwright,624585.0,425794.0,hit_into_play,9,X,1,...,4,Cutter,0,1,1,0,0,1,0,1


In [77]:
# Create new column encoding the information of the previous pitch of the at bat 
# For prev pitch, it should be the same as pitch_name but shifted by down 1
df['prev_pitch'] = df['pitch_name'].shift(1)

# Fill in 'None' where there is a 0-0 count, otherwise leave alone
df['prev_pitch'] = np.where((df['balls'] == 0) & (df['strikes'] == 0), 'None', df['prev_pitch'])

**NOTE**: This approach is inaccurate for the rare cases where a pitcher (or batter) is subbed out mid at-bat.

The above situation only occurs in case of injuries or ejections and are fairly rare. Also could raise issues where rows have been dropped, causing 2 unrelated pitcher/batter combos to be adjacent.
But for the most part I recall the dropped rows came in batches (ie. same game, same year, same player etc) so this should not be a big issues.


The best way I know to address/check for this condition would be a really long for loop (loop through 3.5 mil+ entries).

Elected to not worry about these outlier scenarios for now given how long the for loop takes to run (2hr+) and decided to focus my time on other parts of the project. I tried reading into other ways to do it such as through apply/lambda etc but couldn't figure it out.

In [78]:
# Check the df at this point, primarily to see how the prev_pitch column turned out
# See if it correlates well with the previous pitch of the same at bat
df

,pitch_type,game_date,release_speed,player_name,batter,pitcher,description,zone,type,balls,...,pitch_name,bat_score,fld_score,reg_season,post_season,p_left,p_right,bat_left,bat_right,prev_pitch
0,SI,2015-04-05,90.1,Adam Wainwright,451594.0,425794.0,ball,13,B,0,...,Sinker,0,1,1,0,0,1,1,0,None
1,FC,2015-04-05,88.1,Adam Wainwright,451594.0,425794.0,hit_into_play_no_out,5,X,1,...,Cutter,0,1,1,0,0,1,1,0,Sinker
2,FF,2015-04-05,92.2,Adam Wainwright,624585.0,425794.0,foul,5,S,0,...,4-Seam Fastball,0,1,1,0,0,1,0,1,None
3,CU,2015-04-05,76.6,Adam Wainwright,624585.0,425794.0,blocked_ball,14,B,0,...,Curveball,0,1,1,0,0,1,0,1,4-Seam Fastball
4,FC,2015-04-05,89.9,Adam Wainwright,624585.0,425794.0,hit_into_play,9,X,1,...,Cutter,0,1,1,0,0,1,0,1,Curveball
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615432,FF,2019-10-30,89.1,Zack Greinke,665742.0,425844.0,ball,11,B,0,...,4-Seam Fastball,1,2,0,1,0,1,1,0,None
3615433,CU,2019-10-30,71.1,Zack Greinke,665742.0,425844.0,swinging_strike,11,S,1,...,Curveball,1,2,0,1,0,1,1,0,4-Seam Fastball
3615434,CH,2019-10-30,88.3,Zack Greinke,665742.0,425844.0,ball,13,B,1,...,Changeup,1,2,0,1,0,1,1,0,Curveball
3615435,CH,2019-10-30,88.0,Zack Greinke,665742.0,425844.0,ball,8,B,2,...,Changeup,1,2,0,1,0,1,1,0,Changeup


In [79]:
# Remove the limit of the display to really see how the prev_pitch columns is performing
pd.options.display.max_rows = 999
df.head(150)[['player_name', 'pitch_name', 'type', 'description', 'balls', 'strikes', 'prev_pitch']]

,player_name,pitch_name,type,description,balls,strikes,prev_pitch
0,Adam Wainwright,Sinker,B,ball,0,0,None
1,Adam Wainwright,Cutter,X,hit_into_play_no_out,1,0,Sinker
2,Adam Wainwright,4-Seam Fastball,S,foul,0,0,None
3,Adam Wainwright,Curveball,B,blocked_ball,0,1,4-Seam Fastball
4,Adam Wainwright,Cutter,X,hit_into_play,1,2,Curveball
5,Adam Wainwright,Curveball,B,blocked_ball,0,0,None
6,Adam Wainwright,Cutter,S,swinging_strike,1,0,Curveball
7,Adam Wainwright,4-Seam Fastball,S,called_strike,1,1,Cutter
8,Adam Wainwright,Curveball,B,blocked_ball,1,2,4-Seam Fastball
9,Adam Wainwright,Cutter,S,called_strike,2,2,Curveball


In the above snapshot, it looks like the `prev_pitch` columns aligns well with the actual pitch in the previous row (ie. previous pitch of the at bat). 

The `prev_pitch` column also consistently shows 'None' for the first pitch of the at-bat which is great to see. It seems that for the most part, there were no issues however, I doubt that this turned out perfect and there is likely a handful of errors buried deep within the dataframe.

But considering the sheer size of the dataframe at 3.5 million+ rows, a few errors can be tolerated.

In [80]:
# Drop now that they're unnecessary columns
df.drop(['batter', 'pitcher'], axis = 1, inplace = True)

In [81]:
df['pitch_type'].value_counts()

FF    1291593
SL     582418
FT     398644
CH     376784
SI     302925
CU     297748
FC     201485
KC      88897
FS      54731
KN      11505
IN       6233
EP        868
FO        842
PO        620
SC        113
UN         21
FA         10
Name: pitch_type, dtype: int64

In [82]:
# Redundant to pitch_name
df.drop(['pitch_type'], axis = 1, inplace = True)

In [83]:
# Check the data types
df.dtypes

game_date         object
release_speed    float64
player_name       object
description       object
zone               int32
type              object
balls              int32
strikes            int32
plate_x          float64
plate_z          float64
on_3b              int64
on_2b              int64
on_1b              int64
outs_when_up       int32
inning             int32
sz_top           float64
sz_bot           float64
at_bat_number    float64
pitch_number       int32
pitch_name        object
bat_score          int32
fld_score          int32
reg_season         int64
post_season        int64
p_left             int64
p_right            int64
bat_left           int64
bat_right          int64
prev_pitch        object
dtype: object

In [340]:
# File size reduction from droppping and turning floats to ints: 90 MB (even with a new column added)
# df.to_csv('pitches_finalv3.csv')

***
## More feature engineering


Upon working with the above dataset, I found that there were simply too many different values for `pitch_name` for a model to work with and perform well. 

Here, I will aim to simply that column with the aim of reducing the number of classes my model will need to predict.

My goal is to transform the `4-seam Fastball` and `2-seam Fastball` values and combine them into just one class of `Fastball` since in practice, they are extremely similar pitches and a batter will be able to adjust between them relatively easier.

In [341]:
# Read in the dataframe
df = pd.read_csv('pitches_finalv3.csv', index_col = 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [342]:
df.head()

,game_date,release_speed,player_name,description,zone,type,balls,strikes,plate_x,plate_z,...,pitch_name,bat_score,fld_score,reg_season,post_season,p_left,p_right,bat_left,bat_right,prev_pitch
0,2015-04-05,90.1,Adam Wainwright,ball,13,B,0,0,-1.585,1.720,...,Sinker,0,1,1,0,0,1,1,0,None
1,2015-04-05,88.1,Adam Wainwright,hit_into_play_no_out,5,X,1,0,0.008,2.602,...,Cutter,0,1,1,0,0,1,1,0,Sinker
2,2015-04-05,92.2,Adam Wainwright,foul,5,S,0,0,0.015,2.287,...,4-Seam Fastball,0,1,1,0,0,1,0,1,None
3,2015-04-05,76.6,Adam Wainwright,blocked_ball,14,B,0,1,1.703,0.220,...,Curveball,0,1,1,0,0,1,0,1,4-Seam Fastball
4,2015-04-05,89.9,Adam Wainwright,hit_into_play,9,X,1,2,0.788,1.808,...,Cutter,0,1,1,0,0,1,0,1,Curveball


In [343]:
# Simplify 4S and 2S Fastballs as just 'Fastball'
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Fastball"), 'Fastball', df['pitch_name'])

In [344]:
# See how it worked
df.head()

,game_date,release_speed,player_name,description,zone,type,balls,strikes,plate_x,plate_z,...,pitch_name,bat_score,fld_score,reg_season,post_season,p_left,p_right,bat_left,bat_right,prev_pitch
0,2015-04-05,90.1,Adam Wainwright,ball,13,B,0,0,-1.585,1.720,...,Sinker,0,1,1,0,0,1,1,0,None
1,2015-04-05,88.1,Adam Wainwright,hit_into_play_no_out,5,X,1,0,0.008,2.602,...,Cutter,0,1,1,0,0,1,1,0,Sinker
2,2015-04-05,92.2,Adam Wainwright,foul,5,S,0,0,0.015,2.287,...,Fastball,0,1,1,0,0,1,0,1,None
3,2015-04-05,76.6,Adam Wainwright,blocked_ball,14,B,0,1,1.703,0.220,...,Curveball,0,1,1,0,0,1,0,1,4-Seam Fastball
4,2015-04-05,89.9,Adam Wainwright,hit_into_play,9,X,1,2,0.788,1.808,...,Cutter,0,1,1,0,0,1,0,1,Curveball


In [347]:
# Simplify 4S and 2S Fastballs again, but for the prev_pitch columns
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Fastball"), 'Fastball', df['prev_pitch'])

In [345]:
df['pitch_name'].value_counts()

Fastball            1690247
Slider               582418
Changeup             376784
Sinker               302925
Curveball            297748
Cutter               201485
Knuckle Curve         88897
Split Finger          54731
Knuckle Ball          11505
Intentional Ball       6233
Eephus                  868
Forkball                842
Pitch Out               620
Screwball               113
Unknown                  21
Name: pitch_name, dtype: int64

In [359]:
df['prev_pitch'].value_counts()

Fastball            1265178
None                 929952
Slider               428610
Changeup             273050
Curveball            227939
Sinker               221369
Cutter               148208
Knuckle Curve         67234
Split Finger          39105
Knuckle Ball           8485
Intentional Ball       4427
Eephus                  598
Forkball                595
Pitch Out               592
Screwball                81
Unknown                  14
Name: prev_pitch, dtype: int64

In [349]:
# Save it to another new .csv file
# df.to_csv('pitches_final_fb.csv')

***

After trying to work with the above dataframe, my models still struggled. I decided to simplify ALL the individual pitches into higher level classes.

There are 16 different values within the `pitch_names` column (and correspondingly, the `prev_pitches` column). While each pitcher will only use a subset of these values, the diversity of classes makes predictions much more difficult.

The aim here will be to classify the pitches into the 3 primary high-level pitch classes:

- Fastballs
- Breaking Balls
- Off-speed

**Fastballs** are pitches that are thrown with high velocity and low movement.

**Breaking balls** are pitches which have a high degree of movement (called break) from where they were thrown to where they ultimately cross home plate.

**Off-speed pitches** are pitches that have a moderate degree of movement but are slower than fastballs. Their predominant use is to throw off the batter's timing.

Finally, there is one outlier class called **Junk**.

**Junk** pitches are pitches unlike all others. They are extremely slow pitches, seen very very rarely in games, and have unpredictable directional movement, whereas all other pitches will have a consistent direction of movement.

In [361]:
df = pd.read_csv('pitches_final_fb.csv',index_col= 0)

C:\ProgramData\Anaconda\envs\capstone\lib\site-packages\numpy\lib\arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [362]:
# Lots of different pitches
df['prev_pitch'].value_counts()

Fastball            1265178
None                 929952
Slider               428610
Changeup             273050
Curveball            227939
Sinker               221369
Cutter               148208
Knuckle Curve         67234
Split Finger          39105
Knuckle Ball           8485
Intentional Ball       4427
Eephus                  598
Forkball                595
Pitch Out               592
Screwball                81
Unknown                  14
Name: prev_pitch, dtype: int64

In [363]:
# Test to see if this boolean expression will work
df[df['pitch_name'].str.contains("Fastball|Slider")]['pitch_name'].value_counts()

Fastball    1690247
Slider       582418
Name: pitch_name, dtype: int64

In [364]:
# Simplify classes to fastballs, off-speed, breaking balls, junk and other
# Classification based on https://en.wikipedia.org/wiki/Breaking_ball

# Add Cutters and Sinkers to fastballs
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Cutter|Sinker"), 'Fastball', df['pitch_name'])
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Cutter|Sinker"), 'Fastball', df['prev_pitch'])


# Offspeed
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Changeup|Split Finger|Forkball"), 'Off-speed', df['pitch_name'])
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Changeup|Split Finger|Forkball"), 'Off-speed', df['prev_pitch'])

# Breaking Balls
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Slider|Curveball|Knuckle Curve|Screwball"), 
                            'Breaking Ball', df['pitch_name'])
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Slider|Curveball|Knuckle Curve|Screwball"), 
                            'Breaking Ball', df['prev_pitch'])

# Junk
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Knuckle Ball|Eephus"), 'Junk', df['pitch_name'])
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Knuckle Ball|Eephus"), 'Junk', df['prev_pitch'])

# Other
df['pitch_name'] = np.where(df['pitch_name'].str.contains("Intentional Ball|Pitch Out|Unknown"), 'Other', df['pitch_name'])
df['prev_pitch'] = np.where(df['prev_pitch'].str.contains("Intentional Ball|Pitch Out|Unknown"), 'Other', df['prev_pitch'])

df['pitch_name'].value_counts()

Fastball         2194657
Breaking Ball     969176
Off-speed         432357
Junk               12373
Other               6874
Name: pitch_name, dtype: int64

The classes have now be dramatically condensed relative to where they were before. There is a distinct class imbalance as unsurprisingly, fastballs are the most common pitch used given their simplicity and ubiquitousness.

This class imbalance will have to be considered when creating models down the line.

For now, it appears the data is now finally at the stage where it is clean and ready for models to work with.

In [365]:
# Save to another different .csv file
# df.to_csv('pitches_final_simplified.csv')

***